## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [5]:
from os import path, pardir

## Compile and program project

In [6]:
project_path = [path.curdir, "FFT-S"]
project_path

['.', 'FFT-S']

In [8]:
atprogram(path.abspath(path.join(*project_path)), verbose=2)

Building file: ../../../../shared/gpio_init.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML11E16A__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\arm\cmsis\5.0.1\CMSIS\Include" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/crya" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hri" -I"../trustzone" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\Atmel\SAML11_DFP\1.0.91\include" -I"../../../../shared"  -O1 -ffunction-sections -mlong-calls -g3 -Wall -mcpu=cortex-m23 -c -std=gnu99 -mcmse -MD -MP -MF "gpio_init.d" -MT"gpio_init.d" -MT"gpio_init.o"   -o "gpio_init.o" "../../../../shared/gpio_init.c" 
Finished building: ../../../../shared/gpio_init.c
Building file: .././atmel_start.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program 

0

## Data Logging

In [8]:
live_plot = True

Create a figure for the plot.

In [9]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [9]:
config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "file_name_base": "experiment_fft"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 1,
    "window_title": "Experiment FFT",
}

Stop criteria to pass to the logger:

In [11]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Stop criteria to pass to the parser:

In [12]:
def stop_function(pin_values):
    return all(pin_values)

In [13]:
repetitions = 10

Perform the measurement.

In [ ]:
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
        
charges_allocate = []
charges_fft = []
charges_free = []
times_allocate = []
times_fft = []
times_free = []


with DGILibExtra(**config_dict) as dgilib:
    for _ in range(repetitions):
        dgilib.device_reset()
        dgilib.logger.log(10,stop_fn)
        charge2, time2 = power_and_time_per_pulse(dgilib.data, 2, 0, stop_function=stop_function)
        charge3, time3 = power_and_time_per_pulse(dgilib.data, 3, 0, stop_function=stop_function)
        
        if len(charge2) != 3 or len(time2) != 3 or len(charge3) != 3 or len(time3) != 3:
            print(f"Parsing of measurement data failed, charge2:{charge2}, time2:{time2}, charge3:{charge3}, time3:{time3},")
            break
        charges_allocate.append(charge2[0])
        charges_fft.append(charge3[0])
        charges_free.append(charge2[1])
        times_allocate.append(time2[0])
        times_fft.append(time3[0])
        times_free.append(time2[1])
        # dgilib.logger.plotobj.ax.set_title(f"Previous sample: charge: {sum(charge2+charge3)*1e3:.06} mC, time: {sum(time2+time3):.06} s")
        dgilib.empty_data()

    charges = charges_allocate + charges_fft + charges_switch_to_ifft + charges_ifft + charges_free 
    times = times_allocate + times_fft + times_switch_to_ifft + times_ifft + times_free

    dgilib.logger.plotobj.ax.set_title(f"Average of {repetitions} samples: charge: {sum(charges)/len(charges)*1e3:.06} mC, time: {sum(times)/len(times):.06} s")

In [11]:
import json

config = {}
config["name"] = "FFT 128"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["Allocate", "Free"], 3: ["FFT"]}, 
                      "result_types": ["Charge", "Time"], 
                      "section_types": {"init": ["Allocate FFT"], 
                                        "loop": ["FFT"],
                                        "exit": ["Free FFT"]}}

with open("repeated_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'times': times, 'charges':charges})

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
sum(charges)/repetitions

In [ ]:
print("%.0100f" % (sum(charges)/repetitions))